In [1]:
import math
import pandas as pd
from langdetect import detect
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import CountVectorizer
import math
import matplotlib.pyplot as plt

In [2]:
lem = WordNetLemmatizer() #create lemmatizer

In [3]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shirinharandi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
dictionary = pd.read_csv('../data/processedDict.csv')
dictionary['word'] = dictionary['word'].apply(lambda x: lem.lemmatize(x, pos='n'))

In [5]:
# DELETE LATER
# mask = (reviews['date'] >= '2013-01-01') & (reviews['date'] < '2014-01-01')
# reviews = reviews.loc[mask].copy()
# reviews[:1000]

In [6]:
def get_trends_nice_og(reviews):
    
    def get_unique_words(rev):
        
        allwords = rev['comments'].tolist()
        allwords = " ".join(allwords)
        unique_words = set(allwords.split(' '))

        unique_words = list(unique_words)
        unique_words = [string for string in unique_words if string != ""]

        ls = []
        for word in unique_words:
            word = ''.join([i for i in word if not i.isdigit()])
            ls += [word]
        unique_words= ls
        unique_words = [string for string in unique_words if string != ""]
        unique_words = list(dict.fromkeys(unique_words))
        return unique_words
    
    def get_masked_reviews(start_year, end_year):
        mask = (reviews['date'] >= start_year) & (reviews['date'] < end_year)
        return reviews.loc[mask].copy()
    
#     def get_reviews_list(rev):
        
    
    old_reviews =  get_masked_reviews('2013-01-01', '2014-01-01')
#     print(old_reviews)
    new_reviews =  get_masked_reviews('2017-01-01', '2018-01-01')

    start_year_words = get_unique_words(old_reviews)
    end_year_words = get_unique_words(new_reviews)
    
    old_reviews_list = old_reviews['comments'].tolist()
    new_reviews_list = new_reviews['comments'].tolist()

#     print(new_reviews_list)
    
    def get_freq(word, rev):
        total = 0.0
        rev = list(rev)
        for r in rev:
            r = r.split()
            total += math.log(1 + r.count(word))
        return total

    column_names = ["word", "old_frequency", "new_frequency"]
    out = pd.DataFrame(columns = column_names)
    
    old_total = 0.0
    for word in start_year_words:
        f = get_freq(word, old_reviews_list)
        old_total += f
        out = out.append({'word': word, 'old_frequency': f, "new_frequency": 0.0}, ignore_index=True)
        
    new_total = 0.0
    for word in end_year_words:
        f = get_freq(word, new_reviews_list)
        new_total += f
        if word in start_year_words:
            ind = out.index[out['word']==word]
            out.loc[ind,"new_frequency"] = f
        else:
            out = out.append({'word': word, 'old_frequency': 0.0, 'new_frequency': f}, ignore_index=True)
    
    out["old_sum_count"] = old_total
    out["new_sum_count"] = new_total
    return out

In [7]:
killme = []

In [8]:
def get_trends_nice(reviews):
    
    def get_unique_words(rev):
        
        allwords = rev['comments'].tolist()
        allwords = " ".join(allwords)
        unique_words = set(allwords.split(' '))

        unique_words = list(unique_words)
        unique_words = [string for string in unique_words if string != ""]

        ls = []
        for word in unique_words:
            word = ''.join([i for i in word if not i.isdigit()])
            ls += [word]
        unique_words= ls
        unique_words = [string for string in unique_words if string != ""]
        unique_words = list(dict.fromkeys(unique_words))
        return unique_words
    
    def get_masked_reviews(start_year, end_year):
        mask = (reviews['date'] >= start_year) & (reviews['date'] < end_year)
        return reviews.loc[mask].copy()
    
#     def get_reviews_list(rev):
        
    
    old_reviews =  get_masked_reviews('2013-01-01', '2014-01-01')
#     print(old_reviews)
    new_reviews =  get_masked_reviews('2017-01-01', '2018-01-01')

    start_year_words = get_unique_words(old_reviews)
    end_year_words = get_unique_words(new_reviews)
    
    in_first = set(start_year_words)
    in_second = set(end_year_words)

    in_second_but_not_in_first = in_second - in_first

    total_words_in_reviews = start_year_words + list(in_second_but_not_in_first)
    killme = total_words_in_reviews
    
    old_reviews_list = old_reviews['comments'].tolist()
    new_reviews_list = new_reviews['comments'].tolist()


    def get_freq(word, rev):
        total = 0.0
        rev = list(rev)
        for r in rev:
            r = r.split()
            total += math.log(1 + r.count(word))
        return total

    column_names = ["word", "old_frequency", "new_frequency"]
    out = pd.DataFrame(columns = column_names)
    
    old_total = 0.0
    new_total = 0.0
    for word in total_words_in_reviews:
        f = get_freq(word, old_reviews_list)
        old_total += f
#         print(f)
#         print(word)
        f2 = get_freq(word, new_reviews_list)
        new_total += f2
        out = out.append({'word': word, 'old_frequency': f, "new_frequency": f2}, ignore_index=True)
        
    out["old_sum_count"] = old_total
    out["new_sum_count"] = new_total
    return out

In [9]:
dictionary_words = dictionary['word'].tolist()
dictionary_words = list(set(dictionary_words))
# dictionary_words

In [10]:
cities = ["tokyo", "belize", "santiago", "taipei", "singapore", "beijing", "hong_kong", "mexico_city", "buenos_aires", "rio"]
# cities = ["manchester", "tokyo"]

column_names = ["word", "old_frequency", "new_frequency","old_sum_count", "new_sum_count","city", "Ofrequency", "Nfrequency","freq", "gain", "licia_gain"]
big_out = pd.DataFrame(columns = column_names)

for c in cities: 
    path = "/Users/shirinharandi/Desktop/COMP0031/Data/en_reviews/{}_en.csv".format(c)
    filepath = path
    city_name = c
    reviews = pd.read_csv(filepath)

    table = str.maketrans('', '', string.punctuation) #mapping to strip punctuation in review

    #strip punct of each review -> lemmatise -> output is list of words so join into sentences
    reviews['comments'] = reviews.comments.apply(lambda review: ' '.join(map(str, [lem.lemmatize(word.translate(table), pos='n') for word in review.lower().split()])))
    reviews['date'] = pd.to_datetime(reviews['date'])
    
    out = get_trends_nice_og(reviews)
    
    test = out[out['word'].isin(dictionary_words)]

    test['city'] = city_name
    test['Ofrequency'] = test['old_frequency'] / test['old_sum_count']
    test['Nfrequency'] = test['new_frequency'] / test['new_sum_count']
    test['freq'] = (test['Nfrequency'] + test['Ofrequency'])/2
    test['gain'] = np.where(test['Ofrequency'] == 0, 1000 ,test['Nfrequency'] / test['Ofrequency'])
    test['licia_gain'] = np.where(test['Ofrequency'] == 0, 1000 ,test['new_frequency'] / test['old_frequency'])
    
#     print(test)
    big_out = pd.concat([big_out, test], ignore_index=True) 


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

In [28]:
big_out.to_csv(r'~/Downloads/big_out.csv', index = False)
big_out

,word,old_frequency,new_frequency,old_sum_count,new_sum_count,city,Ofrequency,Nfrequency,freq,gain,licia_gain
0,bedroom,14.203870,516.607518,19587.067856,1.024215e+06,tokyo,0.000725,0.000504,0.000615,0.695556,36.370898
1,centrally,1.386294,129.330841,19587.067856,1.024215e+06,tokyo,0.000071,0.000126,0.000099,1.784124,93.292481
2,quiet,42.281978,3379.334336,19587.067856,1.024215e+06,tokyo,0.002159,0.003299,0.002729,1.528460,79.923752
3,many,23.397105,1600.733519,19587.067856,1.024215e+06,tokyo,0.001195,0.001563,0.001379,1.308384,68.415879
4,mother,5.545177,63.887324,19587.067856,1.024215e+06,tokyo,0.000283,0.000062,0.000173,0.220332,11.521241
...,...,...,...,...,...,...,...,...,...,...,...
3179,arriving,15.249238,28.419034,155587.453652,3.362156e+05,rio,0.000098,0.000085,0.000091,0.862418,1.863636
3180,living,85.322770,192.891917,155587.453652,3.362156e+05,rio,0.000548,0.000574,0.000561,1.046179,2.260732
3181,toiletry,1.386294,11.090355,155587.453652,3.362156e+05,rio,0.000009,0.000033,0.000021,3.702088,8.000000
3182,sympathy,0.000000,0.693147,155587.453652,3.362156e+05,rio,0.000000,0.000002,0.000001,1000.000000,1000.000000


In [12]:
column_names = ["word", "gain_mean"]
mean_gain = pd.DataFrame(columns = column_names)

for dic in dictionary_words:
    dic_rows = big_out[big_out['word'] == dic]
    dic_rows = dic_rows[dic_rows['gain'] != 1000]
    f = dic_rows['gain'].mean()
    mean_gain = mean_gain.append({'word': dic, 'gain_mean': f}, ignore_index=True)

mean_gain

,word,gain_mean
0,bedroom,1.173187
1,cooked,0.775241
2,conveniently,0.929081
3,centrally,2.062954
4,towel,1.115661
...,...,...
316,useful,0.760226
317,home,0.916048
318,room,1.253097
319,fridge,0.743531


In [31]:
top = mean_gain.nlargest(20, 'gain_mean')
bottom = mean_gain.nsmallest(20, 'gain_mean')
top = top.sort_values(by = 'gain_mean', ascending=True)
li = pd.concat([bottom, top], ignore_index=True)
li

,word,gain_mean
0,sympathy,0.130907
1,generosity,0.300439
2,email,0.313008
3,discrete,0.367177
4,punctual,0.367652
5,cabin,0.486632
6,hers,0.491176
7,delightful,0.497754
8,interesting,0.502746
9,waiting,0.503967


In [32]:
main_rows = pd.DataFrame()
for w in li['word']:
#     print(w)
    temp = big_out[big_out['word'] == w]
    temp = temp[temp['gain'] != 1000]
    main_rows = pd.concat([main_rows, temp], ignore_index=True) 

main_rows

,word,old_frequency,new_frequency,old_sum_count,new_sum_count,city,Ofrequency,Nfrequency,freq,gain,licia_gain
0,sympathy,0.693147,0.000000,28749.413958,2.849399e+05,santiago,0.000024,0.000000,0.000012,0.000000,0.000000
1,sympathy,0.693147,0.693147,152972.974940,5.842802e+05,buenos_aires,0.000005,0.000001,0.000003,0.261814,1.000000
2,generosity,0.693147,4.158883,29943.177707,7.425888e+05,taipei,0.000023,0.000006,0.000014,0.241936,6.000000
3,generosity,0.693147,2.772589,18882.041536,4.234295e+05,singapore,0.000037,0.000007,0.000022,0.178372,4.000000
4,generosity,1.386294,12.476649,53601.102349,1.137558e+06,mexico_city,0.000026,0.000011,0.000018,0.424075,9.000000
...,...,...,...,...,...,...,...,...,...,...,...
315,walkable,1.386294,142.330738,18882.041536,4.234295e+05,singapore,0.000073,0.000336,0.000205,4.578372,102.669925
316,walkable,3.465736,168.840230,80611.444300,7.503074e+05,hong_kong,0.000043,0.000225,0.000134,5.234051,48.716993
317,walkable,3.465736,561.397100,53601.102349,1.137558e+06,mexico_city,0.000065,0.000494,0.000279,7.632643,161.984963
318,walkable,15.654703,163.295053,152972.974940,5.842802e+05,buenos_aires,0.000102,0.000279,0.000191,2.731000,10.431054


In [33]:
main_rows.to_csv(r'~/Downloads/main_rows.csv', index = False)

In [34]:
def get_cat_lev1(word):
    ind = dictionary.index[dictionary['word']==word]
    return dictionary.loc[ind, 'cat_lev1'].tolist()[0]

main_rows['cat'] = main_rows['word'].apply(lambda x: get_cat_lev1(x))
main_rows

,word,old_frequency,new_frequency,old_sum_count,new_sum_count,city,Ofrequency,Nfrequency,freq,gain,licia_gain,cat
0,sympathy,0.693147,0.000000,28749.413958,2.849399e+05,santiago,0.000024,0.000000,0.000012,0.000000,0.000000,social
1,sympathy,0.693147,0.693147,152972.974940,5.842802e+05,buenos_aires,0.000005,0.000001,0.000003,0.261814,1.000000,social
2,generosity,0.693147,4.158883,29943.177707,7.425888e+05,taipei,0.000023,0.000006,0.000014,0.241936,6.000000,business
3,generosity,0.693147,2.772589,18882.041536,4.234295e+05,singapore,0.000037,0.000007,0.000022,0.178372,4.000000,business
4,generosity,1.386294,12.476649,53601.102349,1.137558e+06,mexico_city,0.000026,0.000011,0.000018,0.424075,9.000000,business
...,...,...,...,...,...,...,...,...,...,...,...,...
315,walkable,1.386294,142.330738,18882.041536,4.234295e+05,singapore,0.000073,0.000336,0.000205,4.578372,102.669925,business
316,walkable,3.465736,168.840230,80611.444300,7.503074e+05,hong_kong,0.000043,0.000225,0.000134,5.234051,48.716993,business
317,walkable,3.465736,561.397100,53601.102349,1.137558e+06,mexico_city,0.000065,0.000494,0.000279,7.632643,161.984963,business
318,walkable,15.654703,163.295053,152972.974940,5.842802e+05,buenos_aires,0.000102,0.000279,0.000191,2.731000,10.431054,business


In [35]:
main_rows.to_csv(r'~/Downloads/MAIN.csv', index = False)

In [27]:
# DELETE LATER
top['temp'] =  top['word'].apply(lambda x: get_cat_lev1(x))
bottom['temp'] =  bottom['word'].apply(lambda x: get_cat_lev1(x))

In [29]:
big_out.to_csv(r'~/Downloads/big_out.csv', index = False)
